# Working code

In [1]:
import ee
import pandas as pd
from datetime import datetime, timedelta, timezone
import pytz

# Initialize the Earth Engine module with the project ID
ee.Initialize(project='data690-zhouhaomatt')

# Define the points of interest
points_of_interest = [
    (170.89142642028511, -43.99924808088317),
    (170.89155129827327, -43.99924808088317),
    (170.89167617626143, -43.99924808088317),
    (170.89180105424958, -43.99924808088317),
    (170.89192593223774, -43.99924808088317),
    (170.89142642028511, -43.99915824976567),
    (170.89192593223774, -43.99915824976567),
    (170.89142642028511, -43.99906841864817),
    (170.89192593223774, -43.99906841864817)
]

# Convert points to EE objects
ee_points = [ee.Geometry.Point(lon, lat) for lon, lat in points_of_interest]

# Define the central point and buffer for the region of interest (ROI)
central_point = ee.Geometry.Point(170.89167617626143, -43.99906841864817)
roi = central_point.buffer(30)

def get_sentinel_images(start_date, end_date, roi):
    images = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi)
    return images

def get_cloud_score_images(start_date, end_date, roi):
    images = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi)
    return images

def extract_pixel_values(image, points, bands):
    sample = image.select(bands).sampleRegions(
        collection=ee.FeatureCollection(points),
        scale=10,
        geometries=True
    ).getInfo()
    return sample['features']

def convert_timestamp_to_utc_plus_12(timestamp):
    utc_time = datetime.fromtimestamp(timestamp / 1000.0, timezone.utc)
    utc_plus_12_time = utc_time + timedelta(hours=12)
    return utc_plus_12_time.strftime('%Y-%m-%d %H:%M:%S')

def process_data(points, roi, start_date, end_date):
    all_data = []
    bands = ['B2', 'B3', 'B4', 'B8', 'B8A','B11', 'B12']
    cloud_bands = ['cs', 'cs_cdf']
    
    sentinel_images = get_sentinel_images(start_date, end_date, roi)
    cloud_images = get_cloud_score_images(start_date, end_date, roi)
    
    print(f"Processing images from {start_date} to {end_date}")
    sentinel_info = sentinel_images.getInfo()
    
    if not sentinel_info['features']:
        print(f"No Sentinel-2 images found for this date range and region ({start_date} to {end_date}).")
        return pd.DataFrame(all_data)
    else:
        print(f"Found {len(sentinel_info['features'])} Sentinel-2 images.")
    
    for image_info in sentinel_info['features']:
        image = ee.Image(image_info['id'])
        band_values = extract_pixel_values(image, points, bands)
        
        # Extract solar angles from image metadata
        solar_zenith = image_info['properties'].get('MEAN_SOLAR_ZENITH_ANGLE', None)
        solar_azimuth = image_info['properties'].get('MEAN_SOLAR_AZIMUTH_ANGLE', None)
        
        timestamp = image_info['properties']['system:time_start']
        start_time = ee.Date(timestamp)
        end_time = ee.Date(timestamp).advance(1, 'day')
        
        cloud_images_filtered = cloud_images.filterDate(start_time, end_time)
        cloud_info = cloud_images_filtered.getInfo()
        
        if not cloud_info['features']:
            print(f"No Cloud Score images found for the date {datetime.fromtimestamp(timestamp / 1000.0, timezone.utc)} and region.")
            continue
        
        cloud_image = ee.Image(cloud_info['features'][0]['id'])
        cloud_values = extract_pixel_values(cloud_image, points, cloud_bands)
        
        timestamp_str = convert_timestamp_to_utc_plus_12(timestamp)
        
        for i, (band_value, cloud_value) in enumerate(zip(band_values, cloud_values)):
            if band_value['id'] == cloud_value['id']:
                data = {**band_value['properties'], **cloud_value['properties']}
                data['timestamp'] = timestamp_str
                # data['point_index'] = i  # Add point index to differentiate points
                data['solar_zenith'] = solar_zenith
                data['solar_azimuth'] = solar_azimuth
                all_data.append(data)
    
    df = pd.DataFrame(all_data)
    
    # Pivot the dataframe
    df_pivoted = df.pivot(index='timestamp', columns='point_index')
    
    # Flatten the multi-level column index
    df_pivoted.columns = ['_'.join(map(str, col)).strip() for col in df_pivoted.columns.values]
    
    # Reset the index to make timestamp a column
    df_pivoted.reset_index(inplace=True)
    
    return df_pivoted

# Process the data
dataframe = process_data(ee_points, roi, '2018-01-01', '2024-06-18')

# Save or display the dataframe
dataframe.to_csv('sentinel_data_with_cloud_and_solar_angles2.csv', index=False)
print(dataframe.head())


Processing images from 2018-01-01 to 2024-06-18
Found 798 Sentinel-2 images.
No Cloud Score images found for the date 2018-08-27 22:28:23.936000+00:00 and region.
No Cloud Score images found for the date 2018-12-13 22:38:15.731000+00:00 and region.
No Cloud Score images found for the date 2019-03-15 22:28:22.062000+00:00 and region.
No Cloud Score images found for the date 2019-06-23 22:28:29.760000+00:00 and region.
No Cloud Score images found for the date 2019-06-26 22:38:26.585000+00:00 and region.
No Cloud Score images found for the date 2019-06-28 22:28:33.999000+00:00 and region.
No Cloud Score images found for the date 2019-07-01 22:38:30.738000+00:00 and region.
No Cloud Score images found for the date 2019-07-03 22:28:30.473000+00:00 and region.
No Cloud Score images found for the date 2019-07-06 22:38:27.176000+00:00 and region.
No Cloud Score images found for the date 2019-07-08 22:28:34.319000+00:00 and region.
No Cloud Score images found for the date 2019-07-11 22:38:30.90

In [5]:
import ee
import pandas as pd
from datetime import datetime, timedelta, timezone
from concurrent.futures import ThreadPoolExecutor

# Initialize the Earth Engine module with the project ID
ee.Initialize(project='data690-zhouhaomatt')

# Define the points of interest
points_of_interest = [
    (170.89142642028511, -43.99924808088317),
    (170.89155129827327, -43.99924808088317),
    (170.89167617626143, -43.99924808088317),
    (170.89180105424958, -43.99924808088317),
    (170.89192593223774, -43.99924808088317),
    (170.89142642028511, -43.99915824976567),
    (170.89192593223774, -43.99915824976567),
    (170.89142642028511, -43.99906841864817),
    (170.89192593223774, -43.99906841864817)
]

# Convert points to EE objects
ee_points = [ee.Geometry.Point(lon, lat) for lon, lat in points_of_interest]

# Define the central point and buffer for the region of interest (ROI)
central_point = ee.Geometry.Point(170.89167617626143, -43.99906841864817)
roi = central_point.buffer(30)

def get_sentinel_images(start_date, end_date, roi):
    return ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi)

def get_cloud_score_images(start_date, end_date, roi):
    return ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi)

def extract_pixel_values(image, points, bands):
    values = []
    for point in points:
        point_data = image.select(bands).reduceRegion(
            reducer=ee.Reducer.first(),
            geometry=point,
            scale=10
        ).getInfo()
        values.append(point_data)
    return values

def convert_timestamp_to_utc_plus_12(timestamp):
    utc_time = datetime.fromtimestamp(timestamp / 1000.0, timezone.utc)
    utc_plus_12_time = utc_time + timedelta(hours=12)
    return utc_plus_12_time.strftime('%Y-%m-%d %H:%M:%S')

def process_image_data(image_info, points, bands, cloud_bands):
    image = ee.Image(image_info['id'])
    band_values = extract_pixel_values(image, points, bands)

    # Extract solar angles from image metadata
    solar_zenith = image_info['properties'].get('MEAN_SOLAR_ZENITH_ANGLE', None)
    solar_azimuth = image_info['properties'].get('MEAN_SOLAR_AZIMUTH_ANGLE', None)

    timestamp = image_info['properties']['system:time_start']
    timestamp_str = convert_timestamp_to_utc_plus_12(timestamp)

    start_time = ee.Date(timestamp)
    end_time = ee.Date(timestamp).advance(1, 'day')

    cloud_images = get_cloud_score_images(start_time.format('YYYY-MM-dd').getInfo(), end_time.format('YYYY-MM-dd').getInfo(), roi)
    cloud_info = cloud_images.getInfo()

    if not cloud_info['features']:
        return []  # No cloud data for this timestamp

    cloud_image = ee.Image(cloud_info['features'][0]['id'])
    cloud_values = extract_pixel_values(cloud_image, points, cloud_bands)

    data = []
    for i, (band_value, cloud_value) in enumerate(zip(band_values, cloud_values)):
        if band_value and cloud_value:  # Check if both values are valid
            combined_data = {**band_value, **cloud_value}
            combined_data['timestamp'] = timestamp_str
            combined_data['solar_zenith'] = solar_zenith
            combined_data['solar_azimuth'] = solar_azimuth
            combined_data['point_index'] = i  # Add point index to differentiate points
            data.append(combined_data)
    
    return data

def process_data(points, roi, start_date, end_date, max_workers=5):
    all_data = []
    bands = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']
    cloud_bands = ['cs', 'cs_cdf']

    sentinel_images = get_sentinel_images(start_date, end_date, roi)
    sentinel_info = sentinel_images.getInfo()

    if not sentinel_info['features']:
        print(f"No Sentinel-2 images found for this date range and region ({start_date} to {end_date}).")
        return pd.DataFrame(all_data)
    else:
        print(f"Found {len(sentinel_info['features'])} Sentinel-2 images.")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_image_data, image_info, points, bands, cloud_bands) for image_info in sentinel_info['features']]
        for future in futures:
            result = future.result()
            if result:
                all_data.extend(result)
    
    df = pd.DataFrame(all_data)
    
    # Pivot the dataframe
    df_pivoted = df.pivot(index='timestamp', columns='point_index')
    
    # Flatten the multi-level column index
    df_pivoted.columns = ['_'.join(map(str, col)).strip() for col in df_pivoted.columns.values]
    
    # Reset the index to make timestamp a column
    df_pivoted.reset_index(inplace=True)
    
    return df_pivoted

# Process the data
dataframe = process_data(ee_points, roi, '2018-01-01', '2024-06-18')

# Save or display the dataframe
dataframe.to_csv('sentinel_data_with_cloud_and_solar_angles2.csv', index=False)
print(dataframe.head())


Found 798 Sentinel-2 images.
             timestamp  B11_0  B11_1  B11_2  B11_3  B11_4  B11_5  B11_6  \
0  2018-08-28 10:28:23   2263   2267   2267   2279   2279   2263   2279   
1  2018-12-14 10:38:15   3274   3117   3117   3210   3210   3274   3210   
2  2018-12-16 10:28:16   5487   5451   5451   5445   5445   5487   5445   
3  2018-12-19 10:38:13   3879   3888   3888   3890   3890   3879   3890   
4  2018-12-21 10:28:20   2676   2757   2757   2822   2822   2676   2822   

   B11_7  B11_8  ...  solar_zenith_8  solar_azimuth_0  solar_azimuth_1  \
0   2284   2280  ...       61.377174        37.463537        37.463537   
1   3308   3379  ...       31.254377        57.960822        57.960822   
2   5537   5410  ...       32.880865        61.583752        61.583752   
3   3880   3919  ...       31.473882        59.043502        59.043502   
4   2732   2801  ...       33.162840        62.481786        62.481786   

   solar_azimuth_2  solar_azimuth_3  solar_azimuth_4  solar_azimuth_5  \
0 

In [6]:
dataframe

,timestamp,B11_0,B11_1,B11_2,B11_3,B11_4,B11_5,B11_6,B11_7,B11_8,...,solar_zenith_8,solar_azimuth_0,solar_azimuth_1,solar_azimuth_2,solar_azimuth_3,solar_azimuth_4,solar_azimuth_5,solar_azimuth_6,solar_azimuth_7,solar_azimuth_8
0,2018-08-28 10:28:23,2263,2267,2267,2279,2279,2263,2279,2284,2280,...,61.377174,37.463537,37.463537,37.463537,37.463537,37.463537,37.463537,37.463537,37.463537,37.463537
1,2018-12-14 10:38:15,3274,3117,3117,3210,3210,3274,3210,3308,3379,...,31.254377,57.960822,57.960822,57.960822,57.960822,57.960822,57.960822,57.960822,57.960822,57.960822
2,2018-12-16 10:28:16,5487,5451,5451,5445,5445,5487,5445,5537,5410,...,32.880865,61.583752,61.583752,61.583752,61.583752,61.583752,61.583752,61.583752,61.583752,61.583752
3,2018-12-19 10:38:13,3879,3888,3888,3890,3890,3879,3890,3880,3919,...,31.473882,59.043502,59.043502,59.043502,59.043502,59.043502,59.043502,59.043502,59.043502,59.043502
4,2018-12-21 10:28:20,2676,2757,2757,2822,2822,2676,2822,2732,2801,...,33.162840,62.481786,62.481786,62.481786,62.481786,62.481786,62.481786,62.481786,62.481786,62.481786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,2024-06-07 10:28:31,141,241,241,738,738,141,738,126,245,...,72.853098,30.895219,30.895219,30.895219,30.895219,30.895219,30.895219,30.895219,30.895219,30.895219
793,2024-06-10 10:38:28,3071,3019,3019,2925,2925,3071,2925,3094,2985,...,72.248076,28.705300,28.705300,28.705300,28.705300,28.705300,28.705300,28.705300,28.705300,28.705300
794,2024-06-12 10:28:28,1666,1693,1693,1769,1769,1666,1769,1632,1687,...,73.307209,30.958897,30.958897,30.958897,30.958897,30.958897,30.958897,30.958897,30.958897,30.958897
795,2024-06-15 10:38:25,792,646,646,591,591,792,591,718,440,...,72.610559,28.834850,28.834850,28.834850,28.834850,28.834850,28.834850,28.834850,28.834850,28.834850


In [7]:
import ee
import pandas as pd
from datetime import datetime, timedelta, timezone
from concurrent.futures import ThreadPoolExecutor

ee.Initialize(project='data690-zhouhaomatt')

points_of_interest = [
    (170.89142642028511, -43.99924808088317), (170.89155129827327, -43.99924808088317),
    (170.89167617626143, -43.99924808088317), (170.89180105424958, -43.99924808088317),
    (170.89192593223774, -43.99924808088317), (170.89142642028511, -43.99915824976567),
    (170.89192593223774, -43.99915824976567), (170.89142642028511, -43.99906841864817),
    (170.89192593223774, -43.99906841864817)
]

ee_points = [ee.Geometry.Point(lon, lat) for lon, lat in points_of_interest]

central_point = ee.Geometry.Point(170.89167617626143, -43.99906841864817)
roi = central_point.buffer(30)

def get_sentinel_images(start_date, end_date, roi):
    return ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(start_date, end_date).filterBounds(roi)

def get_cloud_score_images(start_date, end_date, roi):
    return ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED').filterDate(start_date, end_date).filterBounds(roi)

def extract_pixel_values(image, points, bands):
    try:
        values = []
        for point in points:
            point_data = image.select(bands).reduceRegion(
                reducer=ee.Reducer.first(),
                geometry=point,
                scale=10
            ).getInfo()
            values.append(point_data)
        return values
    except Exception as e:
        print(f"Error extracting pixel values: {e}")
        return []

def convert_timestamp_to_utc_plus_12(timestamp):
    utc_time = datetime.fromtimestamp(timestamp / 1000.0, timezone.utc)
    utc_plus_12_time = utc_time + timedelta(hours=12)
    return utc_plus_12_time.strftime('%Y-%m-%d %H:%M:%S')

def process_image_data(image_info, points, bands, cloud_bands):
    try:
        image = ee.Image(image_info['id'])
        band_values = extract_pixel_values(image, points, bands)
        solar_zenith = image_info['properties'].get('MEAN_SOLAR_ZENITH_ANGLE', None)
        solar_azimuth = image_info['properties'].get('MEAN_SOLAR_AZIMUTH_ANGLE', None)
        timestamp = image_info['properties']['system:time_start']
        timestamp_str = convert_timestamp_to_utc_plus_12(timestamp)
        start_time = ee.Date(timestamp)
        end_time = start_time.advance(1, 'day')
        cloud_images = get_cloud_score_images(start_time.format('YYYY-MM-dd').getInfo(), end_time.format('YYYY-MM-dd').getInfo(), roi)
        cloud_info = cloud_images.getInfo()
        if not cloud_info['features']:
            return []
        cloud_image = ee.Image(cloud_info['features'][0]['id'])
        cloud_values = extract_pixel_values(cloud_image, points, cloud_bands)
        data = []
        for i, (band_value, cloud_value) in enumerate(zip(band_values, cloud_values)):
            if band_value and cloud_value:
                combined_data = {**band_value, **cloud_value, 'timestamp': timestamp_str, 'solar_zenith': solar_zenith, 'solar_azimuth': solar_azimuth, 'point_index': i}
                data.append(combined_data)
        return data
    except Exception as e:
        print(f"Error processing image data: {e}")
        return []

def process_data(points, roi, start_date, end_date, max_workers=5):
    try:
        all_data = []
        bands = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']
        cloud_bands = ['cs', 'cs_cdf']
        sentinel_images = get_sentinel_images(start_date, end_date, roi)
        sentinel_info = sentinel_images.getInfo()
        if not sentinel_info['features']:
            print(f"No Sentinel-2 images found for this date range and region ({start_date} to {end_date}).")
            return pd.DataFrame(all_data)
        print(f"Found {len(sentinel_info['features'])} Sentinel-2 images.")
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(process_image_data, image_info, points, bands, cloud_bands) for image_info in sentinel_info['features']]
            for future in futures:
                result = future.result()
                if result:
                    all_data.extend(result)
        df = pd.DataFrame(all_data)
        df_pivoted = df.pivot(index='timestamp', columns='point_index')
        df_pivoted.columns = ['_'.join(map(str, col)).strip() for col in df_pivoted.columns.values]
        df_pivoted.reset_index(inplace=True)
        return df_pivoted
    except Exception as e:
        print(f"Error processing data: {e}")
        return pd.DataFrame()

dataframe = process_data(ee_points, roi, '2018-01-01', '2024-06-18')
dataframe.to_csv('sentinel_data_with_cloud_and_solar_angles2.csv', index=False)
print(dataframe.head())

Found 798 Sentinel-2 images.
